In [9]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import re
import os
import json

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras import layers

In [2]:

def clean_text(string: str, 
               punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
               stop_words = stopwords.words('english'),
               # porter = PorterStemmer()
               wnl = WordNetLemmatizer()
              ):
    """
    A method to clean text. It removes punctuations, stop words, applies lemmatization.
    """
    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # stemming/lemmatizing words. That means changing word to its basic format, for example
    # words 'fishing', 'fished', 'fischer' will be changed into a word 'fisch'
    # lemmatization should be better because stemming changes words too much, for example
    # business is changed into busi
    # string = ' '.join([porter.stem(word) for word in string.split()])
    string = ' '.join([wnl.lemmatize(word, pos = "v") for word in string.split()])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string

def create_training_data(tokenizer,
                         max_sen_len,
                         sentences_file,
                         embed_matrix_file,
                         model_folder
                        ):
    """
    Creating a training and testing datasets self.x_train, self.x_test, self.y_train, self.y_test. This function
    also creates and saves a tokenizer and a list of all unique tables names all_unique_values because when we load
    a ready model those values are needed for the 'predict' function.
    """
    sentences_tables = pd.read_excel(sentences_file).values
    random.shuffle(sentences_tables)
    clean_sentences = np.array([clean_text(sentence) for sentence in sentences_tables[:, 0]])

    tokenizer.fit_on_texts(clean_sentences)

    sequences = tokenizer.texts_to_sequences(clean_sentences)
    x = pad_sequences(sequences, maxlen = max_sen_len)

    embed_matrix = pd.read_csv(embed_matrix_file).values

    x_train, x_test = train_test_split(x, test_size = 0.2)

    with open(os.path.join(model_folder, 'tokenizer.json'), 'w') as file:
        json.dump(tokenizer.to_json(), file)
        
    return x_train, x_test


def get_coefs(word, *arr): 
    return word, list(np.asarray(arr, dtype='float'))


def create_embedding_file(tokenizer,
                          embed_file_src = r'model\glove.840B.300d.txt', 
                          embed_file_trg = r'model\model_embeddings.txt'
                         ):
    """
    This function will create an embedding file called embed_file_trg which will contain only those words 
    from embed_file_src which are present in the training dataset. If training dataset wasn't created yet
    then this function will create it.
    """
    # creating a training dataset if we didn't do that yet
    # if not hasattr(self, 'tokenizer'):
    #     self.create_training_data()

    embeddings = dict(get_coefs(*o.split(" ")) for o in open(embed_file_src, errors = 'ignore'))
    with open(embed_file_trg, 'w') as file:
        for word, index in tokenizer.word_index.items():
            word_vector = embeddings[word]
            line = ' '.join(np.concatenate([[word], word_vector]))
            file.write(line + '\n')


def create_embedding_matrix(tokenizer,
                            model_folder,
                            word_vec_dim,
                            embed_file_path,
                           ):
    """
    A function to create the embedding matrix. This is a matrix where each row is a vector representing a word.
    To create that matrix we use a word embedding file which path is equal to embedding_file_path.
    embedding_matrix[row_number] is a vector representation for a word = list(tokenizer.word_index.keys())[row_number - 1]
    First row of embedding_matrix are zeros. This matrix is needed to train a model.
    """
    embeddings = dict(get_coefs(*o.split(" ")) for o in open(embed_file_path, errors = 'ignore'))

    # embedding_matrix[row_number] is a vector representation of a word = self.tokenizer.word_index.keys()[row_number - 1]
    # first row in embedding_matrix is 0
    embedding_matrix = np.zeros((len(tokenizer.word_counts) + 1, word_vec_dim))
    for word, index in tokenizer.word_index.items():
        if index > len(tokenizer.word_counts):
            break
        else:
            try:
                embedding_matrix[index] = embeddings[word]
            except:
                continue

    pd.DataFrame(embedding_matrix).to_csv(os.path.join(model_folder, 'embedding_matrix.csv'))
    return embedding_matrix

In [3]:
tokenizer = Tokenizer()
max_sen_len = 20
sentences_file = r'data\sentences_tables.xlsx'
embed_matrix_file = r'model\embedding_matrix.csv'
model_folder = 'model'
word_vec_dim = 300
embed_file_path = r'model\model_embeddings.txt'

In [4]:
x_train, x_test = create_training_data(
    tokenizer = tokenizer, 
    max_sen_len = max_sen_len,
    sentences_file = sentences_file,
    embed_matrix_file = embed_matrix_file,
    model_folder = model_folder
)

In [5]:
x_train

array([[ 0,  0,  0, ..., 15,  7, 10],
       [ 0,  0,  0, ...,  7, 15, 10],
       [ 0,  0,  0, ...,  3, 11,  1],
       ...,
       [ 0,  0,  0, ..., 12,  9,  1],
       [ 0,  0,  0, ...,  0,  0, 14],
       [ 0,  0,  0, ..., 13, 14, 10]])

In [6]:
embed_matrix = create_embedding_matrix(
    tokenizer = tokenizer,
    model_folder = model_folder,
    word_vec_dim = word_vec_dim,
    embed_file_path = embed_file_path
)

In [7]:
embed_matrix

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-0.50318  ,  0.27905  , -0.045497 , ...,  0.4781   ,  0.13005  ,
        -0.014399 ],
       [-0.89423  ,  0.39636  ,  0.64359  , ..., -0.15076  ,  0.06987  ,
         0.041258 ],
       ...,
       [-0.39054  , -0.55117  , -0.073466 , ...,  0.34569  ,  0.30918  ,
        -0.32873  ],
       [-0.21291  ,  0.22692  , -0.0038332, ..., -0.1319   , -0.37649  ,
         0.26359  ],
       [ 0.37492  , -0.052425 , -0.60094  , ..., -0.36104  , -0.065253 ,
        -0.1206   ]])

In [112]:
class Encoder(Model):
    def __init__(self,
                 vocab_size,
                 embedding_dim,
                 units,
                 batch_size,
                 embed_matrix
                ):
        super().__init__()
        self.units = units
        self.batch_size = batch_size
        self.embedding = layers.Embedding(
            input_dim = embed_matrix.shape[0],
            output_dim = embedding_dim,
            embeddings_initializer = tf.keras.initializers.Constant(embed_matrix)
            # weights = [embed_matrix]
        )
        self.lstm = layers.LSTM(
            units = self.units,
            # return_sequences = True,
            return_state = True
        )
        
    
    def call(self, x, hidden):
        x = self.embedding(x)
        x = tf.reshape(x, [1, x.shape[0], x.shape[1]])
        output, state_h, state_c = self.lstm(x, initial_state = hidden)
        return output, state_h, state_c
    
    def initialize_hidden_state(self):
        state_h = tf.zeros((self.batch_size, self.units))
        state_c = tf.zeros((self.batch_size, self.units))
        return state_h, state_c

In [113]:
encoder = Encoder(vocab_size = 1000, 
                  embedding_dim = 300, 
                  units = 100, 
                  batch_size = 1, 
                  embed_matrix = embed_matrix
                 )

In [114]:
x = np.array([1,2,3])
h, c = encoder.initialize_hidden_state()
encoder(x, [h, c])

(<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
 array([[ 0.22376542,  0.0951889 ,  0.12308526,  0.00936758, -0.09333551,
         -0.00286845,  0.17132208,  0.1383945 ,  0.03012414, -0.00597361,
         -0.1452475 , -0.13623433,  0.28471744,  0.26736513,  0.12396514,
          0.05482598,  0.16541858, -0.06652986,  0.14029554, -0.01865898,
          0.1076088 ,  0.18776111, -0.11111275, -0.07734887, -0.04315203,
          0.01574691,  0.09216573,  0.04126337,  0.0711581 ,  0.14670475,
         -0.08606043, -0.070503  ,  0.1188326 , -0.09154215,  0.1128471 ,
         -0.09936509,  0.17814536, -0.05643369, -0.111694  ,  0.06665881,
         -0.30251756,  0.00483325, -0.17962912,  0.05258976, -0.19325952,
         -0.21569823,  0.00976529,  0.02441533,  0.11729774,  0.02817894,
         -0.27539605,  0.16090515, -0.04335142, -0.2774017 ,  0.02300606,
         -0.10366519,  0.02354721, -0.03674805, -0.02195115, -0.04487948,
          0.30040124, -0.02757289,  0.02317939,  0.26598346, 